# Data Wrangling in Python

### Case Study - Analysis of NYC 311 Service Requests

In this analysis, we will look at the 311 Service Requests dataset published by New York City. 

#### What are 311 Service Requests?

311 Service Requests are requests made by residents for city services such as noise complains, or issues with water and utilities. Each request is recorded along with several pieces of associated information such as the type of request and where in the city the request originated.

#### Dataset
The current dataset includes more than 17 million rows of data and contains data from 2010 to the present.

The full dataset is updated daily and can be accessed [on the NYC Open Data Wesbite](https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9).

#### Analysis

We will look into a small subset of this data: all complaints for the month of April 2018. 

The question we will try to answer with the analysis of the 311 NYC data is as follows:

> What part of the city has the most noise complaints on Sunday?

[comment]: <> (some materials in this notebook are based from Julia Evans https://jvns.ca/blog/2013/12/22/cooking-with-pandas/)

## Part 0 - Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Part 1 - Reading the data
For this demonstration, download the data from the link. 

Once we have read the data, it's a good idea to check the `head`, `shape`, and `info` of the result to make sure all rows and columns have been captured.
- `head` or `tail` will show the first or last few records of your DataFrame
- `shape` will show you the number of rows and columns
- `info` will show you the columns and their respective data types

**Note:** 
- Make sure that the file is in the correct folder or update the path in the read_csv() function to include the correct path to the file. 
- The `pd.read_csv` command has [many options](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) that allow you to control how the file is read. For example, some files will have headers while others won't. Pandas will try to detect these things automatically, but in some cases you may need to play around wiht the options to get it just right.

In [2]:
complaints = pd.read_csv("C_311_Service_Requests_APRIL2018.csv")

/Users/jbsantoalla/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (8,17,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
complaints.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,38837194,04/01/2018 12:00:00 AM,04/10/2018 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Standing Water,Swimming Pool - Unmaintained,1-2 Family Dwelling,10312,45 JEANETTE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.543132,-74.175614,"(40.54313178878882, -74.17561421997694)"
1,38837043,04/01/2018 12:00:00 AM,03/12/2018 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,3+ Family Apt. Building,11217,399 ATLANTIC AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.687177,-73.984820,"(40.687177293988285, -73.98481975668435)"
2,38836824,04/01/2018 12:00:00 AM,04/09/2018 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Commercial Building,11224,2980 WEST 29 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.574249,-73.995169,"(40.57424936932171, -73.99516925322999)"
3,38836823,04/01/2018 12:00:00 AM,03/20/2018 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Apt. Building,10025,360 CENTRAL PARK WEST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.791567,-73.964767,"(40.79156678627206, -73.9647670611818)"
4,38836788,04/01/2018 12:00:00 AM,04/04/2018 06:19:08 PM,DOHMH,Department of Health and Mental Hygiene,Unsanitary Animal Pvt Property,Dog,3+ Family Apartment Building,10454,590 EAST 138 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.806377,-73.916446,"(40.806377143967396, -73.91644582317069)"


## Part 2 - Investigating the data

Now that we know that all our data has been stored in the DataFrame, lets see what columns we want to use, what data is in there, and figure out if we need to clean up the data at all.

The columns we are interested in are:
1. **Created Date** - to determine which day of the week the complaint was filed
2. **Borough** - to determine which part of the city the complaint originated from
3. **Complaint Type** - to determine the category of complaint

Lets take a look at these columns and see what we find there.
- functions like `unique()` and `value_counts()` are useful to investigate columns with string or date values
- functions like `describe()` are useful to investigate columns with numerical values and check for outliers

Some key observations include:
1. There appear to be many different complaint types related to noise like 'Noise - Residential', 'Noise', etc.
    - We will have to group these complaints to get an accurate picture of the overall number of noise complaints
2. There are some complaints that were logged to an 'Unspecified' borough, but the total number of records is low (~4,000 out of total ~200,000)
    - Since we will be grouping our data based on borough we can keep these records
3. The data type of the 'Created Date' is currently a string object, which should be converted to a datetime object to allow us to easily filter on any day of the week, month, etc.

Based on our observations, the cleanup we can do on these columns includes:
1. Redefine 'Created Date' data type for filtering on days of the week (we want the most popular complaint on Sunday).
2. Add a new column *'is_noise'* based on *'Complaint Type'* to capture similar noise-related complaints.

## Part 3 - Cleaning the data

#### 1. Redefine 'Created Date' data type for filtering on days of the week (we want the most popular complaint on Sunday)
- By default, pandas read the 'Created Date' column as a string, so we will tell it that this is a date using the `to_datetime` method.
- We can double-check to make sure it worked by using the `info()` to check for the column's data type

#### 2. Add a new column *'is_noise'* based on *'Complaint Type'* to capture similar noise-related complaints like 'Noise' and 'Noise - Residential'
- Just by looking at the top 10 complaint types using `value_counts`, we notice the data contains redundancy like 'Noise' and 'Noise - Residential', which is surprisingly common. 
- Grouping the data before cleaning it up will skew our results, so we need to clean it up first.
- We will clean this up by identifying all complaints where the type contains the word 'noise' using the `str.contains` method, and creating a new column to indicate whether the complaint is noise-related *'is_noise'*

## Part 4 - Answering the question
Remember, our question is as follows:
> What part of the city has the most noise complaints on Sunday?

Now that our data is clean, all we have to do is:
1. Filter on the noise complaints using our new column 'is_noise'
2. Filter on the days of the week using the 'Created Date' column, which is super easy now that it's defined as a date
    - `dt.dayofweek` returns a different number for each day of the week. Sunday is number 6 as per the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.dayofweek.html#pandas.Series.dt.dayofweek).
3. Group the data by neighbourhood using the 'Borough' column, and sort to show the neighbourhoods at the top of the list.

> Once we do this, it becomes clear that Manhattan has the most noise complaints on Sundays, followed closely by Brooklyn.

## Part 5 - Making clean data available to others
Now that we have cleaned our data, and added a new column, we can write it out to a file for future analysis.

In [5]:
df = pd.read_excel("C_Online_Retail.xlsx")

In [6]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
df.shape

(541909, 8)

In [20]:
df.info(verbose='True')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [21]:
df.define()

AttributeError: 'DataFrame' object has no attribute 'define'

In [22]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
df2 = df.copy()

In [35]:
df2.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [38]:
df2 = df2[(df2['Quantity'] >= 0) & (df2['UnitPrice'] >= 0)]

In [39]:
df2.shape

(531283, 8)

In [40]:
df2.describe()

,Quantity,UnitPrice,CustomerID
count,531283.000000,531283.000000,397924.000000
mean,10.655299,3.898954,15294.315171
std,156.830617,35.876279,1713.169877
min,1.000000,0.000000,12346.000000
25%,1.000000,1.250000,13969.000000
50%,3.000000,2.080000,15159.000000
75%,10.000000,4.130000,16795.000000
max,80995.000000,13541.330000,18287.000000


In [42]:
df2.groupby(['StockCode'])['Quantity'].agg('sum').sort_values(ascending=False)

StockCode
23843     80995
23166     78033
22197     56921
84077     55047
85099B    48478
85123A    41664
84879     36461
21212     36419
23084     31722
22492     26633
22616     26339
21977     24854
22178     24304
15036     23826
17003     23056
21915     22133
22386     21465
23203     20603
20725     19553
22086     19355
84946     19246
47566     18299
84991     18214
22355     18153
22469     17979
20724     17771
85099F    17377
84755     16449
22961     16412
22693     16244
          ...  
90100         1
23628         1
90037A        1
90092         1
23609         1
23602         1
21839         1
85049c        1
85036b        1
85035c        1
47579         1
47591b        1
21410         1
21414         1
84968f        1
46776e        1
46776b        1
90025E        1
20860         1
85031B        1
85035b        1
35609A        1
35600A        1
35597B        1
20849         1
35597A        1
85034b        1
90081B        1
85119         1
m             1
Name: Quantity

In [43]:
df.groupby(['StockCode'])['Quantity'].agg('sum').sort_values(ascending=False)

StockCode
22197      56450
84077      53847
85099B     47363
85123A     38830
84879      36221
21212      36039
23084      30646
22492      26437
22616      26315
21977      24753
22178      23854
17003      23053
15036      22552
21915      22066
22386      21009
23203      20013
20725      18979
84946      18911
22086      18902
84991      18040
47566      18022
22355      18003
22469      17791
20724      17548
84755      16380
22693      16172
22961      16081
85099F     16035
22952      15767
21213      15128
           ...  
79323G      -657
79323S      -674
85172       -755
79341       -905
85036B      -944
72038P      -990
85036C      -999
35610C     -1096
79342B     -1128
79323GR    -1158
D          -1194
23113      -1232
35610A     -1267
23115      -1285
23114      -1294
84598      -1313
23117      -1348
23116      -1362
22351      -1387
23270      -1607
22618      -1632
79323B     -1671
79323P     -2007
23059      -2376
72732      -2472
79323LP    -2618
79323W     -4838
7214